# Tidy Data Exercises

In [1]:
import pandas as pd
import numpy as np

## 1. Attendance Data

Load the `attendance.csv` file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.


Results:
```
name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: grade, dtype: float64

```

In [2]:
attendance = pd.read_csv("attendance.csv")

In [3]:
attendance.head()

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


* P = present (100%)
* T = Tardy (10%)
* A = absent (0%)
* H = Half-day (50%)

In [4]:
attendance = pd.read_csv("attendance.csv")
df = attendance.rename(columns={"Unnamed: 0": "student_name"})
df = df.melt(id_vars= "student_name")
df.variable = pd.to_datetime(df.variable)
df.value = df.value.replace(["P", "A", "T", "H"], [1, 0, .9, .5])
df.rename(columns= ({"variable": "date", "value":"attendance"}), inplace = True)

In [5]:
df.head()

,student_name,date,attendance
0,Sally,2018-01-01,1.0
1,Jane,2018-01-01,0.0
2,Billy,2018-01-01,0.0
3,John,2018-01-01,1.0
4,Sally,2018-01-02,0.9


In [6]:
df.groupby("student_name").attendance.mean()

student_name
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attendance, dtype: float64

## 2. Coffee Levels 

a. Read the `coffee_levels.csv` file.

In [7]:
coffee = pd.read_csv("coffee_levels.csv")

In [8]:
coffee.sample(10)

,hour,coffee_carafe,coffee_amount
25,13,z,0.398520
20,8,z,0.999264
7,15,x,0.215043
15,13,y,0.997464
14,12,y,0.017009
16,14,y,0.058361
19,17,y,0.594126
22,10,z,0.144928
4,12,x,0.898291
3,11,x,0.335533


Transform the data so that each carafe is in it's own column.

In [9]:
df = pd.pivot(coffee, values="coffee_amount", index = "hour", columns="coffee_carafe")

Is this the best shape for the data?

In [10]:
df = df.reset_index()

In [11]:
df.columns.name= ''

In [12]:
df

,hour,x,y,z
0,8,0.816164,0.189297,0.999264
1,9,0.451018,0.521502,0.915990
2,10,0.843279,0.023163,0.144928
3,11,0.335533,0.235529,0.311495
4,12,0.898291,0.017009,0.771947
5,13,0.310711,0.997464,0.398520
6,14,0.507288,0.058361,0.864464
7,15,0.215043,0.144644,0.436364
8,16,0.183891,0.544676,0.280621
9,17,0.391560,0.594126,0.436677


## Cake Recipes

Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures

In [13]:
cake = pd.read_csv("cake_recipes.csv")

In [14]:
cake.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


In [15]:
cake[["recipe", "position"]] = cake["recipe:position"].str.split(":", expand=True)

cake = cake.drop(columns = "recipe:position")

cake = cake.melt(id_vars = ["recipe", "position"])

cake.rename(columns = ({"variable": "temperature", "value": "score"}), inplace=True)

In [16]:
cake.head()

,recipe,position,temperature,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


Which recipe, on average, is the best? recipe b

In [17]:
cake.groupby("recipe").score.mean().nlargest(1)

recipe
b    76.736074
Name: score, dtype: float64

Which oven temperature, on average, produces the best results? 275

In [18]:
cake.groupby("temperature").score.mean().nlargest(1)

temperature
275    74.886754
Name: score, dtype: float64

Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [19]:
cake.groupby(["recipe", "position", "temperature"]).score.mean().nlargest(1)

recipe  position  temperature
b       bottom    300            99.248541
Name: score, dtype: float64